# 7.14 Bernoulli's Theorem-Water

Water at 15 degrees Celsius is flowing through the piping system shown in Crane TP 410M's example at 1500 L/min.

Calculate the velocity in both 4 and 5 inch sizes; and the pressure drop.

Note: This problem suggests to handle the changing size elbow by adding on the result of a smooth expansion, which is also used here.

In [1]:
from math import *
from fluids.units import *
from thermo.units import Chemical

water = Chemical('water', P=2*u.bar, T=15*u.degC)
rho = water.rho
mu = water.mu

Q = 1500*u.L/u.min
r_d = 1.5
_, D1, _, _ = nearest_pipe(Di=100*u.mm)
_, D2, _, _ = nearest_pipe(Di=125*u.mm)
L1 = 34*u.m
L2 = (22+45)*u.m
dH = 22*u.m
beta = D1/D2

V1 = Q/(pi/4*D1**2)
V2 = Q/(pi/4*D2**2)
Re1 = Reynolds(rho=rho, mu=mu, V=V1, D=D1)
Re2 = Reynolds(rho=rho, mu=mu, V=V2, D=D2)
fd1 = friction_factor(Re=Re1, eD=0.0018*u.inch/D1)
fd2 = friction_factor(Re=Re2, eD=0.0018*u.inch/D2)
fd = (fd1+fd2)/2

dP = rho*u.gravity*dH

K_D1 = bend_rounded(Di=D1, angle=90*u.degrees, fd=fd, bend_diameters=r_d)
K_D1 += diffuser_conical(D1, D2, angle=30*u.degrees, fd=fd)
K_D1 += K_from_f(fd=fd1, L=L1, D=D1)

K_D2 = bend_rounded(Di=D2, angle=90*u.degrees, fd=fd, bend_diameters=r_d)
K_D2 += K_from_f(fd=fd2, L=L2, D=D2)

dP += dP_from_K(K=K_D1, rho=rho, V=V1)
dP += dP_from_K(K=K_D2, rho=rho, V=V2)
dP.to(u.Pa)

262809.7611610049 <Unit('pascal')>

The result calculated in Crane's TP 410m is 26450 Pa. Their friction factor is 0.018. Again, it that value is used, the result calculated matches theirs - except this is off by an order of magnitude.

In this edition, the gravitational term was forgotten. The prior 8th edition lists a value of 2.6 bar as the result for this problem. If their friction factor is used with this model, the following calculates a pressure drop of 2.62 bar.

In [2]:
fd = fd1 = fd2 = .018
dP = rho*u.gravity*dH

K_D1 = bend_rounded(Di=D1, angle=90*u.degrees, fd=fd, bend_diameters=r_d)
K_D1 += contraction_round(D1, D2, r_d*D1)
K_D1 += K_from_f(fd=fd1, L=L1, D=D1)

K_D2 = bend_rounded(Di=D2, angle=90*u.degrees, fd=fd, bend_diameters=r_d)
K_D2 += K_from_f(fd=fd2, L=L2, D=D2)

dP += dP_from_K(K=K_D1, rho=rho, V=V1)
dP += dP_from_K(K=K_D2, rho=rho, V=V2)
dP.to(u.Pa)

262439.4308380222 <Unit('pascal')>